In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
data_dir='data'
data_split='val'
classes = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')
n_epochs = 1
emsize = 32
batch_size = 2
margin = 1.

In [3]:
import torch.utils.data as data
from PIL import Image
import os
from datasets import ImageDataset, COCODataset
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from fastai.vision import *

In [4]:
from torchvision import transforms
import torch.nn.functional as F
import torch

def pad_to_size(img: torch.Tensor, size):
    padded = torch.zeros(img.shape[0], size[1], size[0])
    padded[:, :img.shape[1], :img.shape[2]] = img
    return padded

mean, std = 0.1307, 0.3081
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,)),
    lambda x: pad_to_size(x, (512, 512))
])
train_dataset = ImageDataset(
    f"{data_dir}/classy_coconut/train",
    classes,
    tfms=tfms
)
val_dataset = ImageDataset(
    f"{data_dir}/classy_coconut/val",
    classes,
    tfms=tfms
)
n_classes = len(classes)

print(len(train_dataset))
print(len(val_dataset))

67093
14005


In [5]:
# Set up data loaders
from datasets import SiameseImage

siamese_train_dataset = SiameseImage(train_dataset, True) # Returns pairs of images and target same/different
siamese_val_dataset = SiameseImage(val_dataset, False)
# kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
# siamese_train_loader = torch.utils.data.DataLoader(siamese_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
# siamese_dev_loader = torch.utils.data.DataLoader(siamese_dev_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from networks import EmbeddingNet, SiameseNet
from losses import ContrastiveLoss

# cuda = torch.cuda.is_available()
embedding_net = EmbeddingNet(emsize=emsize)
model = SiameseNet(embedding_net)
# if cuda:
#     model.cuda()
loss_fn = ContrastiveLoss(margin)
# lr = 1e-3
# optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
# log_interval = 100

In [6]:
from datasets import SiameseImageList
lls = SiameseImageList.from_datasets(siamese_train_dataset, siamese_val_dataset)

In [7]:
dbunch = (lls
          .transform(get_transforms())
          .databunch(bs=batch_size, device=0, num_workers=8)
#           .normalize((mean, std))
         )

In [8]:
learner = Learner(dbunch, model, loss_func=loss_fn)

In [ ]:
learner.fit_one_cycle(n_epochs, 1e-3)
learner.save("asdf")